In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
print(torch.__version__)

1.4.0


In [35]:
with open('datasets.txt', 'r', encoding='utf-8') as f:
    datasets = f.readlines()
    datasets = ''.join(datasets).split('\n')

In [36]:
docs = [data.split(' ') for data in datasets]

In [38]:
tokens = [t for doc in docs for t in doc]

In [41]:
uniq_tokens = list(set(tokens))

In [62]:
token2vec = {token:vec for token, vec in zip(uniq_tokens, np.eye(len(uniq_tokens)))}
ix2token = {vec.argmax():token for token, vec in 
            zip(uniq_tokens, np.eye(len(uniq_tokens)))}

In [101]:
class NPLM(nn.Module):
    
    def __init__(self, VOCAB_SIZE, FEATURE_VECTOR_SIZE, HIDDEN_SIZE=32):
        super(NPLM, self).__init__()
        self.VOCAB_SIZE = VOCAB_SIZE
        self.HIDDEN_SIZE = HIDDEN_SIZE
        # projection layer (linear)
        self.proj_layer = nn.Linear(VOCAB_SIZE, FEATURE_VECTOR_SIZE, bias=False)
        # hidden layer (non-linear)
        self.hidden_layer = nn.Linear(FEATURE_VECTOR_SIZE, HIDDEN_SIZE, bias=True)
        # output layer
        self.output_layer = nn.Linear(HIDDEN_SIZE, VOCAB_SIZE, bias=True)
        
    def forward(self, x):
        x = self.proj_layer(x)
        x = torch.tanh(self.hidden_layer(x))
        x = self.output_layer(x)
        return x

In [102]:
net = NPLM(len(uniq_tokens), 100)

In [103]:
net

NPLM(
  (proj_layer): Linear(in_features=1009, out_features=100, bias=False)
  (hidden_layer): Linear(in_features=100, out_features=32, bias=True)
  (output_layer): Linear(in_features=32, out_features=1009, bias=True)
)

In [104]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

5
torch.Size([100, 1009])


In [105]:
params

[Parameter containing:
 tensor([[ 0.0146, -0.0159,  0.0211,  ...,  0.0264,  0.0148,  0.0240],
         [-0.0049, -0.0193, -0.0209,  ..., -0.0087, -0.0274, -0.0091],
         [ 0.0309,  0.0227, -0.0100,  ..., -0.0041,  0.0278, -0.0281],
         ...,
         [ 0.0066,  0.0188,  0.0010,  ..., -0.0053,  0.0271,  0.0142],
         [-0.0197,  0.0185,  0.0274,  ...,  0.0155,  0.0085, -0.0012],
         [ 0.0251, -0.0215, -0.0268,  ...,  0.0038,  0.0298,  0.0173]],
        requires_grad=True), Parameter containing:
 tensor([[-0.0759,  0.0057,  0.0638,  ...,  0.0636, -0.0993,  0.0426],
         [-0.0357,  0.0384, -0.0809,  ...,  0.0799, -0.0884,  0.0217],
         [-0.0756,  0.0711, -0.0133,  ...,  0.0607,  0.0302, -0.0936],
         ...,
         [-0.0474, -0.0217, -0.0249,  ...,  0.0639,  0.0803, -0.0315],
         [ 0.0881,  0.0205,  0.0734,  ..., -0.0998, -0.0073,  0.0608],
         [ 0.0960, -0.0905,  0.0543,  ...,  0.0480,  0.0858,  0.0599]],
        requires_grad=True), Parameter conta

In [106]:
inputs = torch.from_numpy(np.array(list(map(lambda x: token2vec[x], docs[0]))))

In [107]:
net = net.float()
outputs = net(inputs.float())

In [108]:
outputs

tensor([[-0.0467, -0.1317, -0.1607,  ..., -0.1367, -0.0280, -0.0172],
        [-0.0395, -0.1245, -0.1563,  ..., -0.1436, -0.0347, -0.0160],
        [-0.0469, -0.1398, -0.1474,  ..., -0.1426, -0.0257, -0.0019],
        [-0.0523, -0.1185, -0.1538,  ..., -0.1470, -0.0148, -0.0074],
        [-0.0486, -0.1191, -0.1553,  ..., -0.1491, -0.0208, -0.0119]],
       grad_fn=<AddmmBackward>)

In [109]:
net.zero_grad()

In [110]:
params = list(net.parameters())
params

[Parameter containing:
 tensor([[ 0.0146, -0.0159,  0.0211,  ...,  0.0264,  0.0148,  0.0240],
         [-0.0049, -0.0193, -0.0209,  ..., -0.0087, -0.0274, -0.0091],
         [ 0.0309,  0.0227, -0.0100,  ..., -0.0041,  0.0278, -0.0281],
         ...,
         [ 0.0066,  0.0188,  0.0010,  ..., -0.0053,  0.0271,  0.0142],
         [-0.0197,  0.0185,  0.0274,  ...,  0.0155,  0.0085, -0.0012],
         [ 0.0251, -0.0215, -0.0268,  ...,  0.0038,  0.0298,  0.0173]],
        requires_grad=True), Parameter containing:
 tensor([[-0.0759,  0.0057,  0.0638,  ...,  0.0636, -0.0993,  0.0426],
         [-0.0357,  0.0384, -0.0809,  ...,  0.0799, -0.0884,  0.0217],
         [-0.0756,  0.0711, -0.0133,  ...,  0.0607,  0.0302, -0.0936],
         ...,
         [-0.0474, -0.0217, -0.0249,  ...,  0.0639,  0.0803, -0.0315],
         [ 0.0881,  0.0205,  0.0734,  ..., -0.0998, -0.0073,  0.0608],
         [ 0.0960, -0.0905,  0.0543,  ...,  0.0480,  0.0858,  0.0599]],
        requires_grad=True), Parameter conta

In [131]:
F.softmax(outputs, dim=1).argmax(dim=1)

tensor([781, 781, 781, 781, 781])